# Search

> Set of Core Functions that are shared across modules

In [ ]:
#| default_exp search

In [ ]:
#| hide
from nbdev.showdoc import *

## API Requests and Headers

First, we need a way to easily set the header information for requests to the Copper API. I'm envisioning a process something like

```python
import cu_api

cuapi_wrapper.set_headers(os.environ('COPPER_KEY'), 'my-cool-email@gmail.com')
# rest of code...
```

To do this, let's make a system that takes in the key pieces of header information and greats global variable with all of the header information. Other functions then can use this global var.

In [ ]:
#| export
import requests, time
from cu_api import core, config
from cu_api.query import _process_query
from tqdm.autonotebook import tqdm

In [ ]:
#| export
def _search_loop(search_query, # Instance of Query object
                 url, # Copper API url,
                 max_retries:int = 5, # Maximum retry attempts
                 retry_delay:float = 2.0, # Delay in seconds between retries,
                 **kwags
                ):
    """Standard search loop used across all Copper record types
    """
    if 'debug' in kwags.keys(): debug = kwags['debug']
    else:                       debug = False

    if 'tqmd' in kwags.keys(): tqmd_msg = f"Searching Copper: {kwags['tqmd']}"
    else:                      tqmd_msg = "Searching Copper"

    Sess = core.get_session()

    Native_Params, CF_Parms, Outputs  = _process_query(search_query)
    if debug: print(f"Native_Params: {Native_Params}")
    if debug: print(f"CF_Params: {CF_Parms}")

    total_pages = page = 1
    combined_results = []
    
    while page <= total_pages:
        page_params = {
            "page_size": 100,
            "page_number": page,
            }
        
        if Native_Params: page_params.update(Native_Params)

        if CF_Parms:     page_params.update({"custom_fields":CF_Parms}) 
        if debug: print(f'Payload: {page_params}')

        retries = 0
        while retries < max_retries:
            result = Sess.post(url,json=page_params)
    
            if result.status_code == 200:
                total_pages = (int(result.headers['X-PW-TOTAL'])//100)+1
                
                # Creatig Progress Bar:
                if page == 1: progress_bar = tqdm(total=total_pages,desc=tqmd_msg)
                progress_bar.update(1)  # Update the progress bar
                
                result_json = result.json()
                combined_results.extend(result_json)
                page +=1
                break
            elif result.status_code == 429:
                retries += 1
                #print(f"Rate limit encountered. Retry {retries}/{max_retries} in {retry_delay} seconds.")
                time.sleep(retry_delay)
            else:
                print(f"Issue with page {page}")
                raise Exception(f"Error {result.status_code} retrieved from API")
    
    progress_bar.close()  # Close the progress bar when done
    
    return combined_results, Outputs


In [ ]:
#| export

def search_over_field(field:str,
                      url:str,
                      search_query):
    
    combined_results = []
    list_names = search_query[field]

    for value in list_names:
        search_query[field] = value

        name_results, Outputs = _search_loop(search_query= search_query, url= url, tqmd= f"'{field}' is '{value}'")
        combined_results.extend(name_results)
    return combined_results, Outputs

In [ ]:
show_doc(_search_loop)

---

[source](https://github.com/cooper-richason/cu_api/blob/main/cu_api/search.py#L13){target="_blank" style="float:right; font-size:smaller"}

### _search_loop

>      _search_loop (search_query, url, max_retries:int=5,
>                    retry_delay:float=2.0, **kwags)

*Standard search loop used across all Copper record types*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| search_query |  |  | Instance of Query object |
| url |  |  | Copper API url, |
| max_retries | int | 5 | Maximum retry attempts |
| retry_delay | float | 2.0 | Delay in seconds between retries, |
| kwags |  |  |  |

In [ ]:
#| export
def get_owners(assignee_ids:list)-> dict:
    Sess = core.get_session()
    assignee_dict = {}

    for userid in assignee_ids:
        results = Sess.get(f"https://api.copper.com/developer_api/v1/users/{int(userid)}")
        assignee_dict[userid] = results.json().get('name', None)

    return assignee_dict

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()